In [2]:
from pydub import AudioSegment
location_videofile = "C:/Users/Razai/one_to_one_Trim15.mp4"
mp4_file = location_videofile
wav_filename = "textnew16hz11.wav"
track = AudioSegment.from_file(mp4_file,  format= 'mp4')
file_handle = track.export(wav_filename, format='wav')


mp3_audio = AudioSegment.from_file(r"textnew16hz11.wav", format="wav")
new_mp3_audio= mp3_audio + 10 # increase volume of audio
print(len(mp3_audio)/(1000*60))

# 12 Minutes audio breaks into 3 minutes 4 audio files (slicing is done by milliseconds)

counter_audio = 180
split_audio = [mp3_audio[:180*1000]]
for i in range(4):
    split_audio.append(mp3_audio[counter_audio*1000:(counter_audio+180)*1000])
    counter_audio += 180

count = 0
# # lets save it!
for count, audio_object in enumerate(split_audio):
    count += 1
    with open(f"{count}_textnew16hz11.wav", 'wb') as out_f:
        audio_object.export(out_f, format='wav')

print(mp3_audio.frame_rate) 

0.25315
48000


In [3]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [4]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
collection_of_text = []
for i in range(4):

    speech, rate = librosa.load(f"{i+1}_textnew16hz11.wav" , sr=16000)

    input_values = processor(speech, return_tensors='pt').input_values
    # Store logits (non-normalized predictions)
    with torch.no_grad():
        logits = model(input_values).logits

    # Store predicted id's
    predicted_ids = torch.argmax(logits, dim=-1)
    # decode the audio to generate text
    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = processor.batch_decode(predicted_ids)[0]
    # transcriptions = tokenizer.decode(predicted_ids[0])
    print(transcription)
    collection_of_text.append(transcription)

print(collection_of_text)
final_complete_speech = ""

# convert batch of text into one complete sentence
for i in collection_of_text:
    final_complete_speech += i

print(final_complete_speech)

with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(final_complete_speech) 

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


HLLO HELLO HOW ARE YOU I'M FINE THANK YOU AND YOU I'M FINE THANK YOU WHERE ARE YOU FROM I COME FROM ENGLAND


ValueError: Input signal length=0 is too small to resample from 48000->16000

In [ ]:
from scipy.io.wavfile import read as read_wavs
sampling_rate, data=read_wavs("C:/Users/Razai/textnew16hz11.wav")
print (sampling_rate)